<a href="https://colab.research.google.com/github/tubagokhan/ADGM/blob/main/ADGMDocParser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install python-docx

In [36]:
from docx import Document
import json
import random
import string

# Load the DOCX file
doc = Document("/content/ModelFriendlyCOBS.docx")

# Function to generate a unique 6-digit identifier
def generate_unique_identifier():
    return ''.join(random.choices('ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789', k=6))

# Modified generate_paragraph_number function to handle 'List (a)'
def generate_paragraph_number(style_name, last_number, is_list_a=False, is_description=False, is_description_item=False):

    if is_list_a:
        parts = last_number.split('.')
        if not parts[-1].isdigit():  # If the last part is already a letter
            letter = parts[-1][-1]  # Get the last character, which should be a letter
            next_letter = chr(ord(letter) + 1) if letter != 'z' else 'a'  # Increment the letter
            parts[-1] = parts[-1][:-1] + next_letter  # Replace the last letter with the next letter
        else:
            parts.append('a')  # If the last part is a digit, add 'a'
        return '.'.join(parts)
    elif is_description:
        # Handle Description numbering
        parts = last_number.split('.')
        if len(parts)>2:
          number=str(parts[0])+'.'+parts[1]+'.'+'D'
          return number
        else:
          parts.append('D')
        return '.'.join(parts)
    elif is_description_item:
        # Handle Description numbering
        print(last_number)
        parts = last_number.split('.')
        if parts[-1]=='D':  # If the last part is already a letter
            parts.append('1')
        else :
            next_number=str(int(parts[-1]) + 1)
            parts[-1] = parts[-1][:-1] + next_number

        return '.'.join(parts)
    else:
        parts = last_number.split('.')
        if style_name == 'Heading 1':
            return str(int(parts[0]) + 1) if parts[0].isdigit() else '1'
        elif style_name == 'Heading 2':
            if len(parts) < 2: parts.append('0')
            parts[0] = str(int(parts[0]))
            parts[1] = str(int(parts[1]) + 1)
            return '.'.join(parts[:2])
        elif style_name in ['Heading 3', 'List Paragraph', 'Normal']:
            if len(parts) < 3: parts.extend(['0'] * (3 - len(parts)))
            parts[0] = str(int(parts[0]))
            parts[1] = str(int(parts[1]))
            parts[2] = str(int(parts[2]) + 1)
            return '.'.join(parts[:3])
        elif style_name == 'Description':
            return '.'.join(parts[:2])

# Function to add a new section with the required structure
def add_new_section(paragraph_number, heading, sections, text=None):
    section = {
        "unique_identifier": generate_unique_identifier(),
        "paragraph_number": paragraph_number,
        "heading": heading,
        "text": text if text else "null",
        "subparagraphs": []
    }
    sections.append(section)
    return section

# Initialize the JSON structure
nested_json = {
    "title": "",
    "DocumentID": generate_unique_identifier(),
    "Chapters": []
}

# Variables to track the current state
current_chapter = None
current_section = None
current_subsection = None
paragraph_number = "0"

# Iterate through paragraphs in the DOCX file
for paragraph in doc.paragraphs:
    style_name = paragraph.style.name
    text = paragraph.text.strip()
    if style_name == 'Title':
        nested_json["title"] = text

    elif style_name in ['Heading 1', 'Heading 2', 'Heading 3', 'List Paragraph', 'Normal' ]:
        paragraph_number = generate_paragraph_number(style_name, paragraph_number)

        if style_name == 'Heading 1':
            current_chapter = add_new_section(paragraph_number=paragraph_number, heading=text, sections=nested_json["Chapters"])
            current_section = None
            current_subsection = None
        elif style_name == 'Heading 2':
            current_section = add_new_section(paragraph_number=paragraph_number, heading=text, sections=current_chapter["subparagraphs"])
            current_subsection = None
        elif style_name == 'Heading 3':
            current_subsection = add_new_section(paragraph_number=paragraph_number, heading=None, text=text, sections=current_section["subparagraphs"] if current_section else current_chapter["subparagraphs"])
        elif style_name in ['List Paragraph', 'Normal']:
            add_new_section(paragraph_number=paragraph_number, heading=None, sections=current_section["subparagraphs"] if current_section else current_chapter["subparagraphs"], text=text)

    elif style_name == 'Description':
        paragraph_number = generate_paragraph_number(style_name, paragraph_number, is_description=True)
        current_section = add_new_section(paragraph_number=paragraph_number, heading=text, sections=current_section["subparagraphs"] if current_section else current_chapter["subparagraphs"], text=None)
        # Keep a reference to the current 'Description' section to add 'Description List Items' as subparagraphs
        last_description_section = current_section

    elif style_name == 'Description List Item':
        # Ensure there's a current 'Description' section to add to
        if last_description_section:
            paragraph_number = generate_paragraph_number(style_name, last_number=paragraph_number, is_description_item=True)
            # Add the 'Description List Item' as a subparagraph to the last 'Description' section
            add_new_section(paragraph_number=paragraph_number, heading=None, sections=last_description_section["subparagraphs"], text=text)
        else:
            print("Warning: 'Description List Item' found without preceding 'Description' section")


    elif style_name == 'List (a)':
        paragraph_number = generate_paragraph_number(style_name, paragraph_number, is_list_a=True)
        if current_subsection:
            add_new_section(paragraph_number=paragraph_number, heading=None, sections=current_subsection["subparagraphs"], text=text)
        elif current_section:
            add_new_section(paragraph_number=paragraph_number, heading=None, sections=current_section["subparagraphs"], text=text)
        elif current_chapter:
            add_new_section(paragraph_number=paragraph_number, heading=None, sections=current_chapter["subparagraphs"], text=text)

# Save the JSON structure to a file
with open("/content/final_nested_cobs.json", "w") as json_file:
    json.dump(nested_json, json_file, indent=4)


2.1.D
2.1.D.1


In [39]:
from docx import Document
import json
import random
import string

# Constants
STYLE_TITLE = 'Title'
STYLE_HEADING_1 = 'Heading 1'
STYLE_HEADING_2 = 'Heading 2'
STYLE_HEADING_3 = 'Heading 3'
STYLE_LIST_PARAGRAPH = 'List Paragraph'
STYLE_NORMAL = 'Normal'
STYLE_DESCRIPTION = 'Description'
STYLE_DESCRIPTION_LIST_ITEM = 'Description List Item'
STYLE_LIST_A = 'List (a)'
ALPHANUMERIC = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'

# Load the DOCX file
doc = Document("/content/ModelFriendlyCOBS.docx")

def generate_unique_identifier():
    return ''.join(random.choices(ALPHANUMERIC, k=6))

def generate_paragraph_number(style_name, last_number, is_list_a=False, is_description=False, is_description_item=False):
    parts = last_number.split('.')
    if is_list_a:
        if parts[-1].isdigit():
            parts.append('a')
        else:
            letter = parts[-1][-1]
            next_letter = chr(ord(letter) + 1) if letter != 'z' else 'a'
            parts[-1] = parts[-1][:-1] + next_letter
    elif is_description:
        if len(parts) > 2:
            parts = parts[:2]  # Keep only the first two parts
        parts.append('D')
    elif is_description_item:
        if parts[-1] == 'D':
            parts.append('1')
        else:
            parts[-1] = str(int(parts[-1]) + 1)
    else:
        increment_index = 0 if style_name == STYLE_HEADING_1 else 1 if style_name == STYLE_HEADING_2 else 2
        while len(parts) <= increment_index:
            parts.append('0')
        parts[increment_index] = str(int(parts[increment_index]) + 1)
        parts = parts[:increment_index + 1]
    return '.'.join(parts)

def add_new_section(paragraph_number, heading="", text="null", sections=[]):
    section = {
        "unique_identifier": generate_unique_identifier(),
        "paragraph_number": paragraph_number,
        "heading": heading,
        "text": text,
        "subparagraphs": []
    }
    sections.append(section)
    return section

nested_json = {"title": "", "DocumentID": generate_unique_identifier(), "Chapters": []}
current_chapter = current_section = current_subsection = last_description_section = None
paragraph_number = "0"

for paragraph in doc.paragraphs:
    style_name = paragraph.style.name
    text = paragraph.text.strip()

    if style_name == STYLE_TITLE:
        nested_json["title"] = text
    elif style_name in [STYLE_HEADING_1, STYLE_HEADING_2, STYLE_HEADING_3, STYLE_LIST_PARAGRAPH, STYLE_NORMAL]:
        paragraph_number = generate_paragraph_number(style_name, paragraph_number)
        if style_name == STYLE_HEADING_1:
            current_chapter = add_new_section(paragraph_number, text, sections=nested_json["Chapters"])
            current_section = current_subsection = None
        elif style_name == STYLE_HEADING_2:
            current_section = add_new_section(paragraph_number, text, sections=current_chapter["subparagraphs"])
            current_subsection = None
        elif style_name == STYLE_HEADING_3:
            current_subsection = add_new_section(paragraph_number, text=text, sections=current_section["subparagraphs"] if current_section else current_chapter["subparagraphs"])
        else:
            add_new_section(paragraph_number, text=text, sections=current_section["subparagraphs"] if current_section else current_chapter["subparagraphs"])
    elif style_name == STYLE_DESCRIPTION:
        paragraph_number = generate_paragraph_number(style_name, paragraph_number, is_description=True)
        last_description_section = add_new_section(paragraph_number, text, sections=current_section["subparagraphs"] if current_section else current_chapter["subparagraphs"])
    elif style_name == STYLE_DESCRIPTION_LIST_ITEM and last_description_section:
        paragraph_number = generate_paragraph_number(style_name, paragraph_number, is_description_item=True)
        add_new_section(paragraph_number, text=text, sections=last_description_section["subparagraphs"])
    elif style_name == STYLE_LIST_A:
        paragraph_number = generate_paragraph_number(style_name, paragraph_number, is_list_a=True)
        sections = current_subsection["subparagraphs"] if current_subsection else current_section["subparagraphs"] if current_section else current_chapter["subparagraphs"]
        add_new_section(paragraph_number, text=text, sections=sections)

with open("/content/final_nested_cobs.json", "w") as json_file:
    json.dump(nested_json, json_file, indent=4)
